# Advanced Text Generation Techniques and Tools<

In [ ]:
# !pip install langchain-community
# !pip install llama-cpp-python

# Loading an LLM

In [ ]:
# !wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

In [ ]:
from langchain_community.llms import LlamaCpp

# Initialize the LlamaCpp language model with specified parameters
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",  # Path to the GGUF model file
    n_gpu_layers=-1,       # Use all available GPU layers
    max_tokens=500,        # Maximum number of tokens to generate in a response
    n_ctx=2048,            # Context window size
    seed=42,               # Random seed for reproducibility
    verbose=False          # Disable verbose output
)

Let's invoke the LlamaCpp language model with a prompt directly:

In [2]:
# Directly invoke the LlamaCpp language model with a prompt
# This sends the prompt string to the model and returns its response
llm.invoke("Hi! My name is Hatef. What is 2 times 5?")

''

### Chains

### Building Complex Workflows with Multiple Chains

Now that we've mastered basic prompt templating, let's explore how to create more sophisticated workflows by chaining multiple operations together. This approach allows us to break down complex tasks into manageable steps, where each step builds upon the previous one.

**Why Chain Multiple Prompts?**
- **Modularity**: Each step handles a specific task
- **Reusability**: Components can be used in different combinations  
- **Quality**: Focused prompts often produce better results than complex single prompts
- **Debugging**: Easier to identify and fix issues in individual steps

We'll demonstrate this by building a creative story generation pipeline that:
1. Creates a compelling title from a summary
2. Develops character descriptions based on the title and summary
3. Generates the complete story using all previous components

**Modern vs. Legacy Approaches**: You'll notice we show both the deprecated `LLMChain` approach and the modern `RunnableSequence` method. The modern approach is preferred for new projects.

In [ ]:
from langchain import PromptTemplate

# Define a prompt template for single-turn LLM interaction.
# The template uses the variable "input_prompt" to insert user input.
template = """<|user|>
{input_prompt}<|end|>
<|assistant|>"""

# Create a PromptTemplate object with the template and input variable.
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

# Create a basic chain by piping the prompt template into the LlamaCpp language model.
# This chain takes an input prompt, formats it using the template, and sends it to the LLM for a response.
basic_chain = prompt | llm

# Use the chain
basic_chain.invoke(
    {"input_prompt": "Hi! My name is Hatef. What is 2 times 5?"})

' Hi Hatef! 2 times 5 is equal to 10.'

**Output Analysis**: Notice how the model generates creative, contextually appropriate titles. The modern pipe operator (`|`) creates a clean, readable chain that's easier to debug and maintain than the deprecated `LLMChain` approach.

### Title Generation Chain
Create Title Generator


In [ ]:
from langchain import PromptTemplate

# Create a chain for the title of our story
template = """<|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title_chain = title_prompt | llm
title_chain.invoke({"summary": "A man and his tiger"})
# Example Output: "The Unbreakable Bond: A Man and His Tiger"

' The Lone Guardian: A Tale of Man and Tigress.'

Next, let's create a character description generator that uses both the original summary and the generated title.

In [12]:
# Create a chain for the character description using the summary and title
template = """<|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character_chain = character_prompt | llm

**Chain Dependencies**: Notice how the character generator depends on both the original summary and the generated title. This demonstrates how chains can have multiple inputs from different sources - a key pattern in complex workflows.

Now let's create the final story generator that combines all our components.

In [13]:
# Create a chain for the story using the summary, title, and character description
template = """<|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story_chain = story_prompt | llm

**Manual Chain Management**: The above approach gives us full control over the data flow between steps. However, LangChain provides more elegant solutions for sequential operations.

Let's explore the modern way to create this same pipeline using `RunnableSequence` and helper functions.

In [17]:
# Step 1: Generate the title
title = title_chain.invoke({"summary": "A man and his tiger"})

# Step 2: Generate the character description
character = character_chain.invoke({"summary": "A man and his tiger", "title": title})

# Step 3: Generate the story
story = story_chain.invoke({"summary": "A man and his tiger", "title": title, "character": character})

print(story)


 In the heart of the dense, emerald jungles lived a man named John, whose unyielding love for animals was renowned far and wide. Among his cherished companions was Kali, an extraordinary tiger with piercing golden eyes that mirrored his profound wisdom. Forbidden by society's norms, their bond had flourished into something truly unique; a testament to the resilience of two spirits intertwined in mutual respect and understanding. Embarking on a remarkable journey together, they defied boundaries as John tended to Kali with unwavering compassion, while she offered him an unspoken strength that surpassed all human comprehension. Their story, "The Bonded Journey: A Man and His Tiger," became an epitome of the extraordinary connection possible between mankind and nature when approached with pure love and respect.


In [16]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

# The llm object is already defined above

# 1. Title prompt: Generates a story title given a summary
title_prompt = PromptTemplate(
    template="<|user|>Create a title for a story about {summary}. Only return the title.<|end|><|assistant|>",
    input_variables=["summary"],
)
title_chain = title_prompt | llm

# 2. Character prompt: Describes the main character using summary and title
character_prompt = PromptTemplate(
    template="<|user|>Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|><|assistant|>",
    input_variables=["summary", "title"],
)
character_chain = character_prompt | llm

# 3. Story prompt: Generates the story using summary, title, and character description
story_prompt = PromptTemplate(
    template="<|user|>Create a story about {summary} with the title {title}. The main character is: {character}. Only return the story and it cannot be longer than one paragraph<|end|><|assistant|>",
    input_variables=["summary", "title", "character"],
)
story_chain = story_prompt | llm

# 4. Helper function: Adds the generated title to the input dictionary
def with_title(inputs):
    title = title_chain.invoke({"summary": inputs["summary"]})
    return {**inputs, "title": title}

# Helper function: Adds the generated character description to the input dictionary
def with_character(inputs):
    character = character_chain.invoke({"summary": inputs["summary"], "title": inputs["title"]})
    return {**inputs, "character": character}

# Compose the pipeline: sequentially generates title, character, and story
llm_chain = (
    RunnableLambda(with_title)
    | RunnableLambda(with_character)
    | RunnableLambda(lambda x: story_chain.invoke({
        "summary": x["summary"],
        "title": x["title"],
        "character": x["character"]
      }))
)

# Test the chain with a sample summary
result = llm_chain.invoke({"summary": "A man and his tiger"})
print(result)


 In "Unleashed Bonds: The Tale of Man and the Majestic Tiger," we follow Samuel, a dedicated wildlife enthusiast whose life takes an unexpected turn when he rescues a breathtakingly beautiful but captive-bound tiger from poachers. Forging a profound connection with this majestic creature, named Shadow by its striking midnight fur and piercing gaze, Samuel finds himself embarking on an extraordinary journey that transcends the boundaries of human understanding. As they traverse through a world marred by misunderstanding towards their kind, their unwavering bond faces challenges from societal norms to legal constraints, but together they champion compassion and conservation, illuminating the strength found in the heartbeats between species. Samuel's courage and Shadow's grace become symbols of hope for a future where man and beast can coexist harmoniously, their tale inspiring generations about the unleashed bonds that define true companionship.


# Memory

**The Core Issue: LLMs Don't Remember**

- **LLMs are stateless** — they have no memory of previous conversations
- Each prompt is treated as a completely new interaction
- Information shared in one prompt is forgotten in the next

**Example**

In [17]:
# Let's give the LLM our name
basic_chain.invoke({"input_prompt": "Hi! My name is Hatef. What is 2 times 5?"})

' Hi Hatef! 2 times 5 is equal to 10.'

In [18]:
# Next, we ask the LLM to reproduce the name
basic_chain.invoke({"input_prompt": "What is my name?"})

" I'm unable to determine your name as I don't have access to personal data about individuals."

**The Problem:** Poor user experience, no conversational continuity

**Three Memory Types in LangChain:**

1. **ConversationBufferMemory** — Store complete conversation history
2. **ConversationBufferWindowMemory** — Keep only last k conversations
3. **ConversationSummaryMemory** — Summarize conversation history

**How Memory Works:**

- **Extends prompt template** to include conversation history
- **Automatically manages** chat context between interactions
- **Enables continuity** in multi-turn conversations

**Trade-offs to Consider:**

- **Speed vs. Memory** — More history = slower processing
- **Accuracy vs. Efficiency** — Complete history vs. compression

## ConversationBuffer

**How It Works:**

- Stores **entire conversation history** in memory
- Appends **full chat context** to each new prompt
- **No information loss** within context window limits

**Implementation:**

In [19]:
# Create an updated prompt template to include a chat history
template = """<|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

**Key Feature:** `chat_history` variable holds all previous conversations


In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [25]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Hatef. What is 2 times 5?"})

{'input_prompt': 'Hi! My name is Hatef. What is 2 times 5?',
 'chat_history': '',
 'text': " Hello Hatef! 2 times 5 equals 10.\n\nHere's a quick breakdown:\n\n- You have two numbers, 2 and 5.\n- The operation is multiplication (times).\n- Multiply the first number by the second one: 2 * 5 = 10.\n- Thus, the answer to your question is 10."}

In [26]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Hatef. What is 2 times 5?\nAI:  Hello Hatef! 2 times 5 equals 10.\n\nHere's a quick breakdown:\n\n- You have two numbers, 2 and 5.\n- The operation is multiplication (times).\n- Multiply the first number by the second one: 2 * 5 = 10.\n- Thus, the answer to your question is 10.",
 'text': ' Your name is the one I have been provided, which in this case is "AI" or Artificial Intelligence.'}

## ConversationBufferMemoryWindow

In [27]:
from langchain.memory import ConversationBufferWindowMemory

# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

/var/folders/fd/9ctr8dys3tv3jyfrxn6_8y5m0000gp/T/ipykernel_25205/535028085.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [ ]:
# Ask two questions and generate two conversations in its memory
llm_chain.invoke({"input_prompt":"Hi! My name is Hatef and I am 85 years old. What is 2 times 5?"})
llm_chain.invoke({"input_prompt":"What is 4 + 3?"})

{'input_prompt': 'What is 3 + 3?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  Hello Maarten, it's nice to meet you! The answer to 1 + 1 is 2.\n\nHowever, if you have any other questions or need further assistance, feel free to ask!",
 'text': " Hello again! 3 + 3 equals 6. If there's anything else I can help you with, just let me know!"}

In [ ]:
# Check whether it knows the name we gave it
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  Hello Maarten, it's nice to meet you! The answer to 1 + 1 is 2.\n\nHowever, if you have any other questions or need further assistance, feel free to ask!\nHuman: What is 3 + 3?\nAI:  Hello again! 3 + 3 equals 6. If there's anything else I can help you with, just let me know!",
 'text': ' Your name is Maarten.'}

In [28]:
# Check whether it knows the age we gave it
llm_chain.invoke({"input_prompt":"What is my age?"})

{'input_prompt': 'What is my age?',
 'chat_history': '',
 'text': " As a platform, I'm unable to know personal information about you such as your age. It's essential for privacy reasons."}

What Happened: Age from first interaction was forgotten because only last 2 conversations kept

## ConversationSummary

In [29]:
# Create a summary prompt template
summary_prompt_template = """<|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [30]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

/var/folders/fd/9ctr8dys3tv3jyfrxn6_8y5m0000gp/T/ipykernel_25205/4216441337.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [31]:
# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Hatef. What is 2 times 5?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': ' Hatef introduced themselves and asked the AI for the result of 2 times 5, which the AI confirmed as 10. The simple calculation provided further confirms that 2 multiplied by 5 equals 10.',
 'text': " I don't have the capability to recall personal information about individuals unless it has been shared with me in the course of our conversation. Therefore, I cannot know your name."}

In [32]:
# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

{'input_prompt': 'What was the first question I asked?',
 'chat_history': ' Hatef initiated the conversation and requested the AI to calculate 2 times 5, which was confirmed by the AI as 10. Subsequently, when asked about their name, the AI clarified its inability to recall personal information shared within the ongoing dialogue.',
 'text': ' The first question you asked was: "calculate 2 times 5."'}

In [33]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_history': ' Hatef started the conversation and asked the AI to calculate 2 times 5, resulting in an answer of 10. Later, when inquiring about their name, the AI reminded that it cannot recall personal information shared during the current chat session. When prompted about the initial query, the AI confirmed it was "calculate 2 times 5."'}

# Agents

In [ ]:
import os
from langchain_openai import ChatOpenAI

# Load OpenAI's LLMs with LangChain
os.environ["OPENAI_API_KEY"] = "MY_KEY"
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [ ]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
I need to find the current price of a MacBook Pro in USD first before converting it to EUR.
Action: duckduck
Action Input: "current price of MacBook Pro in USD"[snippet: View at Best Buy. The best MacBook Pro overall The MacBook Pro 14-inch with the latest M3-series chips offers outstanding, best-in-class performance while getting fantastic battery life and ..., title: The best MacBook Pro in 2024: our picks for the top Pro models, link: https://www.techradar.com/best/best-macbook-pro], [snippet: Starts at $1,299. Upgradable to 24 GB of memory and 2 TB of storage. 67W USB-C charger included. The M2-powered MacBook Pro is available now for a starting price of $1,299 on Apple's website ..., title: MacBook Pro 13-inch (M2, 2022) review | Tom's Guide, link: https://www.tomsguide.com/reviews/macbook-pro-13-inch-m2-2022], [snippet: The late-2023 MacBook Pro update also marks the demise of the 13-inch MacBook Pro, which has been replaced by a 14-inch mo

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'The current price of a MacBook Pro in USD is $2,249.00. It would cost approximately 1911.65 EUR with an exchange rate of 0.85 EUR for 1 USD.'}